### Load env variables

In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

openai_api_key = os.getenv("OPENAI_API_KEY")
pinecone_api_key = os.getenv("PINECONE_API_KEY")

In [ ]:
### Test connection to OpenAI API via api-key

### Init LLM
# llm = ChatOpenAI(
#     temperature=0,
#     openai_api_key=openai_api_key,
#     model="gpt-3.5-turbo",
# )

### Ask question and get answer
# q = "What is AI?"
# a = llm.invoke(q)
# print(a)

# Large Language Model as a tool for extracting knowledge from text
Idea from [pjaskulski](https://github.com/pjaskulski/gpt_historical_text/blob/main/README.md)

Because his work is based on data that is not publicly available ([src1](https://rcin.org.pl/ihpan/dlibra/publication/4137#description) [src2](https://fbc.pionier.net.pl/details/nnWR8W7)), I will use wikipedia article as a source.

## Download wikipedia dataset ([huggingface source](https://huggingface.co/datasets/wikipedia))
As stated in source, we need to install `mwparserfromhell` for lib to be able to parse loaded data. 
I specified also newest dump of polish wikipedia ([20240420](https://dumps.wikimedia.org/plwiki/20240420/)) to be downloaded.

In [8]:
%pip install -qU mwparserfromhell

Note: you may need to restart the kernel to use updated packages.


In [2]:
from datasets import load_dataset

data = load_dataset("wikipedia", language="pl", date="20240420", split='train[:10000]', num_proc=4)
data

Extracting data files: 100%|██████████| 1/1 [00:00<00:00, 296.54it/s]
Generating train split: 0 examples [00:00, ? examples/s]

Extracting content fromExtracting content fromExtracting content from Extracting content from  /Users/priv/.cache/huggingface/datasets/downloads/c16fb72f3265b9811c5c7db5044de02c35b8fe0da562bfdfbd6d578cef260a64/Users/priv/.cache/huggingface/datasets/downloads/0c0a0496e0aea38989c77b2041e031a2f26e4e4190f75d2acb752bb73c197b4b 
/Users/priv/.cache/huggingface/datasets/downloads/e2a700b16446a0fa7699a3ddabc3af1ac82e5e53412706833f3b060958e0f7c7
/Users/priv/.cache/huggingface/datasets/downloads/74da23b795d370b9cee643f4fb72c9628300470ef7082c422a2f9909333dd850



Generating train split: 455000 examples [09:13, 929.64 examples/s] 

Extracting content from /Users/priv/.cache/huggingface/datasets/downloads/ed6e18ad2c0299372b5f8dd5cc5baaff7a84c931bdf44adcdf6899ae0213f0f5


Generating train split: 686000 examples [13:05, 908.80 examples/s] 

Extracting content from /Users/priv/.cache/huggingface/datasets/downloads/972c6800931999319fb2b8f1b2d9dcb5fcfa855bb03c6a1e0ece78368d45db1b


Generating train split: 1236064 examples [27:38, 492.70 examples/s]

Extracting content from /Users/priv/.cache/huggingface/datasets/downloads/ec52f45c9e6e47f0770a2a2bd6513f7376d082675cdd8b6c7f8ce009eae4e39a


Generating train split: 1611471 examples [51:48, 518.35 examples/s]


Dataset({
    features: ['id', 'url', 'title', 'text'],
    num_rows: 10000
})

Get all data and cache it (save to local disk, sync with git-lfs (see `data_cache/dataset/info.md`))

In [5]:
data2 = load_dataset("wikipedia", language="pl", date="20240420", num_proc=4)
data2 = data2["train"]

/Users/priv/dev/M.Sc._AI/.venv/lib/python3.12/site-packages/datasets/table.py:1421: FutureWarning: promote has been superseded by promote_options='default'.
  table = cls._concat_blocks(blocks, axis=0)


In [16]:
data2

Dataset({
    features: ['id', 'url', 'title', 'text'],
    num_rows: 1611471
})

In [17]:
data2.save_to_disk("data_cache/dataset/wikipediaPL")

Saving the dataset (7/7 shards): 100%|██████████| 1611471/1611471 [00:07<00:00, 212601.21 examples/s]


## Load dataset from local cache

In [4]:
from datasets import load_from_disk

data = load_from_disk("./data_cache/dataset/wikipediaPL")

/Users/priv/dev/M.Sc._AI/.venv/lib/python3.12/site-packages/datasets/table.py:1421: FutureWarning: promote has been superseded by promote_options='default'.
  table = cls._concat_blocks(blocks, axis=0)


In [5]:
data

Dataset({
    features: ['id', 'url', 'title', 'text'],
    num_rows: 1611471
})

## Get article and ask LLM to generate knowledge graph

In [6]:
data.to_csv("temp/wikipl.csv")

Creating CSV from Arrow format: 100%|██████████| 1612/1612 [00:48<00:00, 33.34ba/s]


2992727740

In [9]:
data.to_csv("temp/a.csv", columns=["url", "title"])

Creating CSV from Arrow format: 100%|██████████| 1612/1612 [00:09<00:00, 177.18ba/s]


124401939

In [10]:
index = 23 #Agatha Christie
data[index]

{'id': '32',
 'url': 'https://pl.wikipedia.org/wiki/Agatha%20Christie',
 'title': 'Agatha Christie',
 'text': "Agatha Christie, lady Mallowan, DBE, właśc. Agatha Mary Clarissa Miller Christie (ur. 15 września 1890 w Torquay, zm. 12 stycznia 1976 w Wallingford) – brytyjska autorka powieści kryminalnych i obyczajowych.\n\nNajbardziej znana na świecie pisarka kryminałów oraz najlepiej sprzedająca się autorka wszech czasów. Wydano ponad miliard egzemplarzy jej książek w języku angielskim oraz drugi miliard przetłumaczonych na 45 języków obcych. Pod pseudonimem Mary Westmacott wydała kilka powieści obyczajowych, które również cieszyły się popularnością.\n\nChristie wydała ponad 90 powieści i sztuk teatralnych. Ich akcja toczyła się głównie w zamkniętych pomieszczeniach, a mordercą mógł być tylko jeden z mieszkańców. Wniosła wiele nowych rozwiązań do tradycyjnej sztuki pisania kryminałów. Niektóre z jej powieści mają zaskakujące i wcześniej niespotykane w kryminałach rozwiązania, np. Zabójst